In [1]:
#I used the code on huggingface github

from transformers import RobertaTokenizer, IBertModel
import torch
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer

def tokenize_function(examples):
    return tokenizer(examples["text"])

model_checkpoint = "distilroberta-base"
datasets = load_dataset("text", data_files={"train": 'sample.txt', "validation": 'sample.txt'})
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)



Using custom data configuration default-2627d976c71275ba
Reusing dataset text (/home/doogers/.cache/huggingface/datasets/text/default-2627d976c71275ba/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)










In [2]:
datasets["train"][10]

{'text': '41 79 3 Quarter, 41 62 3 Dotted Quarter, 42 55 3 Dotted Quarter, 43 47 3 Triplet Sixteenth.'}

In [4]:
tokenized_datasets["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [0, 3897, 2929, 112, 8424, 90, 5310, 33326, 4, 2]}

In [5]:


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


block_size = tokenizer.model_max_length

In [6]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [7]:
tokenizer.decode(lm_datasets["train"][10]["input_ids"])

' 113 Dotted Sixteenth.</s><s>41 61 113 Dotted Eighth, 41 77 113 Eighth, 42 70 113 Dotted Eighth, 43 55 113 Dotted Sixteenth.</s><s>41 84 114 Half, 41 74 114 Eighth, 42 64 114 Dotted Quarter, 43 55 114 Dotted Sixteenth.</s><s>42 70 114 Eighth, 43 55 114 Dotted Sixteenth.</s><s>41 77 115 Half, 41 72 115 Eighth, 42 75 115 Half, 43 53 115 Dotted Sixteenth.</s><s>41 82 115 Eighth.</s><s>41 69 116 Dotted Eighth, 41 87 116 Dotted Eighth, 42 60 116 Half, 43 53 116 Dotted Sixteenth.</s><s>41 75 117 Sixteenth, 42 72 117 Sixteenth, 43 41 117 Sixteenth.</s><s>41 63 118 Sixteenth, 42 51 118 Sixteenth, 43 51 118 Sixteenth.</s><s>41 68 120 Sixteenth, 41 76 120 Sixteenth, 42 59 120 Sixteenth, 43 52 120 Dotted Eighth.</s><s>41 72 120 Sixteenth, 41 76 120 Sixteenth, 42 57 120 Sixteenth.</s><s>43 40 121 Half.</s><s>43 45 121 Eighth.</s><s>41 69 122 Triplet Sixteenth, 41 64 122 Dotted Sixteenth, 43 48 122 Half.</s><s>41 72 122 Dotted Sixteenth, 42 57 122 Eighth.</s><s>41 64 123 Dotted Eighth, 41 71 123 D

In [8]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [9]:
from transformers import Trainer, TrainingArguments

In [10]:
training_args = TrainingArguments(
    "test-clm",
    num_train_epochs = 30,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [12]:
trainer.train()

/home/doogers/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


/home/doogers/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [31]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [33]:
model.save_pretrained('model_saved2')